### Gas demand correlations ###

Using NGrid data from Hazel (Thornton et al., 2019) for exploring the decadal relationship between different climate predictors (surface variables, indicies etc.) and UK gas demand (which has been postprocessed to remove non-linear trends).

In [ ]:
# Local libraries
import os
import sys
import glob

# Third-party libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import pearsonr

These data are in .dat format, so we will need to convert this into .csv with NaNs using Pandas.

In [ ]:
# Print the files in the directory
dir = "/home/users/benhutch/NGrid_demand"

# List the files in this directory
os.listdir(dir)

### Data description ###

#### Electricity demand ####

* *Electricity_demand_5yrRmean_full_GB.dat* - daily UK electricity demand data with the 5-year running mean removed including data for weekends and holidays.
* *Electricity_demand_5yrRmean_full_nonhol_GB.dat* - daily UK electricity demand data with the 5-year running mean removed *not* including data for weekends and holidays.

#### Gas demand ####

* *Gas_raw_demand_GB.dat* - daily gas demand for the UK.
* *Gas_demand_5yrRmean_full_GB.dat* - daily gas demand for the UK with the 5-year running mean removed.
* *Gas_demand_5yrRmean_full_nonhol_GB.dat* - daily gas demand for the UK with the 5-year running mean removed *not* including data for weekends and holidays.

In [ ]:
# We want to load these all into a .csv file
raw_gas_file = "Gas_raw_demand_GB.dat"

# load using pandas
df_raw_gas = pd.read_csv(os.path.join(dir, raw_gas_file), sep=" ", header=0)

In [ ]:
df_raw_gas.head()

In [ ]:
# Try loading the data with NaN values
gas_file_nohols = "Gas_demand_5yrRmean_full_nonhol_GB.dat"

# Load in the .csv file
df_gas_nohols = pd.read_csv(os.path.join(dir, gas_file_nohols), sep=" ", header=0, na_values="NA")

In [ ]:
df_gas_nohols.head()

In [ ]:
# laod the other data
gas_demand_with_hols = "Gas_demand_5yrRmean_full_GB.dat"

# Load in the .csv file
df_gas_hols = pd.read_csv(os.path.join(dir, gas_demand_with_hols), sep=" ", header=0, na_values="NA")


In [ ]:
df_gas_hols.head()

In [ ]:
# Convert the date to a datetime object
df_gas_hols["date"] = pd.to_datetime(df_gas_hols["date"], format="%Y-%m-%d")


In [ ]:
df_gas_hols

In [ ]:
# Convert the date to a datetime object
df_gas_nohols["date"] = pd.to_datetime(df_gas_nohols["date"], format="%Y-%m-%d")

In [ ]:
# Same for the raw data
df_raw_gas["date"] = pd.to_datetime(df_raw_gas["date"], format="%Y-%m-%d")

In [ ]:
# For nohols add the suffix _nohols to the demand_full_5yrRmean column
df_gas_nohols.rename(columns={"demand_full_5yrRmean": "demand_full_5yrRmean_nohols"}, inplace=True)

In [ ]:
# join the three dataframes on the date column
df_gas = pd.merge(df_gas_hols, df_gas_nohols, on="date")

# merge this with the raw data
df_gas = pd.merge(df_gas, df_raw_gas, on="date")

In [ ]:
df_gas.head()

In [ ]:
# rename demand_MCM as gas_demand_raw
df_gas.rename(columns={"demand_MCM": "gas_demand_raw"}, inplace=True)

# rename demand_full_5yrRmean as gas_demand_5yrRmean
df_gas.rename(columns={"demand_full_5yrRmean": "gas_demand_5yrRmean"}, inplace=True)

# rename demand_full_5yrRmean_nohols as gas_demand_5yrRmean_nohols
df_gas.rename(columns={"demand_full_5yrRmean_nohols": "gas_demand_5yrRmean_nohols"}, inplace=True)

In [ ]:
df_gas.head()

In [ ]:
# Load in the electricity demand data
elec_file = "Electricity_demand_5yrRmean_full_nonhol_GB.dat"

# Load in the .csv file
df_elec_nohols = pd.read_csv(os.path.join(dir, elec_file), sep=" ", header=0, na_values="NA")

In [ ]:
df_elec_nohols.head()

In [ ]:
# rename demand_full_5yrRmean as elec_demand_5yrRmean_nohols
df_elec_nohols.rename(columns={"demand_full_5yrRmean": "elec_demand_5yrRmean_nohols"}, inplace=True)

In [ ]:
# laod the other data
elec_demand_with_hols = "Electricity_demand_5yrRmean_full_GB.dat"

# Load in the .csv file
df_elec_hols = pd.read_csv(os.path.join(dir, elec_demand_with_hols), sep=" ", header=0, na_values="NA")

In [ ]:
df_elec_hols.head()

In [ ]:
# Convert the date to datetime
df_elec_hols["date"] = pd.to_datetime(df_elec_hols["date"], format="%Y-%m-%d")

In [ ]:
# Convert the date to datetime
df_elec_nohols["date"] = pd.to_datetime(df_elec_nohols["date"], format="%Y-%m-%d")

In [ ]:
# Merge the two dataframes on the date column
df_elec = pd.merge(df_elec_hols, df_elec_nohols, on="date")

In [ ]:
df_elec

In [ ]:
# rename demand_full_5yrRmean as elec_demand_5yrRmean
df_elec.rename(columns={"demand_full_5yrRmean": "elec_demand_5yrRmean"}, inplace=True)

In [ ]:
# Merge the two dataframes on 'date' column
df_merged = df_elec.merge(df_gas, on='date', how='left')

In [ ]:
df_merged

In [ ]:
# set the filename
output_file = "gas_electricity_demand_data.csv"

# output directory
output_dir = "/home/users/benhutch/NGrid_demand/csv_files"

# if the directory does not exist, create it
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# save the dataframe to a .csv file
df_merged.to_csv(os.path.join(output_dir, output_file), index=False)